# Интеллектуальный анализ данных – весна 2024
# Домашнее задание 6: классификация текстов

Правила:



*   Домашнее задание оценивается в 10 баллов.
*   Можно использовать без доказательства любые результаты, встречавшиеся на лекциях или семинарах по курсу, если получение этих результатов не является вопросом задания.
*  Можно использовать любые свободные источники с *обязательным* указанием ссылки на них.
*  Плагиат не допускается. При обнаружении случаев списывания, 0 за работу выставляется всем участникам нарушения, даже если можно установить, кто у кого списал.
*  Старайтесь сделать код как можно более оптимальным. В частности, будет штрафоваться использование циклов в тех случаях, когда операцию можно совершить при помощи инструментов библиотек, о которых рассказывалось в курсе.

В этом домашнем задании вам предстоит построить классификатор текстов.

Будем предсказывать эмоциональную окраску твиттов о коронавирусе.



In [ ]:
import numpy as np
import pandas as pd
from typing import  List
import matplotlib.pyplot as plt
import seaborn as sns
from string import punctuation

In [1]:
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


X_train = np.array([[2, 3], [3, 1], [1, 2], [2, 1], [1, 2]])
y_train_1 = np.array([4, 0.5, -0.3, -2.3, 0.1])
y_train_2 = np.array([10, 20, 20, 10, 15])
y_train_3 = np.array([25, -1, 15, 10, 20])

# Тестовая выборка
X_test = np.array([[-4, 1], [2, 2], [2, 3]])
y_test_1 = np.array([-2, -0.1, 1.9])
y_test_2 = np.array([20, 15, 5])
y_test_3 = np.array([20, 15, 20])

# (1) Евклидова метрика, Y1, K = 2, MSE
model = KNeighborsRegressor(n_neighbors=2, metric='euclidean')
model.fit(X_train, y_train_1)
y_pred_1 = model.predict(X_test)
mse_1 = mean_squared_error(y_test_1, y_pred_1)

# (2) Евклидова метрика, Y3, K = 1, R2
model = KNeighborsRegressor(n_neighbors=1, metric='euclidean')
model.fit(X_train, y_train_3)
y_pred_3 = model.predict(X_test)
r2_3 = r2_score(y_test_3, y_pred_3)

# (3) Манхэттенская метрика, Y2, K = 3, RMSE
model = KNeighborsRegressor(n_neighbors=3, metric='manhattan')
model.fit(X_train, y_train_2)
y_pred_2 = model.predict(X_test)
rmse_2 = np.sqrt(mean_absolute_error(y_test_2, y_pred_2))

print("MSE для первой задачи:", mse_1)
print("R2 для второй задачи:", r2_3)
print("RMSE для третьей задачи:", rmse_2)


MSE для первой задачи: 2.471666666666667
R2 для второй задачи: -8.0
RMSE для третьей задачи: 2.23606797749979


In [ ]:
df = pd.read_csv('/content/tweets_coronavirus.csv', encoding='latin-1')
df.sample(4)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
10147,16111,61063,Australia,21-03-2020,Seriously frustrated. The same person who has ...,Extremely Negative
11081,17246,62198,"Los Angeles, CA",21-03-2020,Are you on the frontlines fighting #coronaviru...,Extremely Negative
11606,17877,62829,"East Midlands, UK",21-03-2020,#Coronavirus: Three arrested after police find...,Extremely Negative
16761,24204,69156,Republic of Bobistan,25-03-2020,Top 5 victims of 5 Common sense 4 Grocery stor...,Extremely Positive


Для каждого твитта указано:


*   UserName - имя пользователя, заменено на целое число для анонимности
*   ScreenName - отображающееся имя пользователя, заменено на целое число для анонимности
*   Location - местоположение
*   TweetAt - дата создания твитта
*   OriginalTweet - текст твитта
*   Sentiment - эмоциональная окраска твитта (целевая переменная)



## Задание 1 Подготовка (0.5 балла)

Целевая переменная находится в колонке `Sentiment`.  Преобразуйте ее таким образом, чтобы она стала бинарной: 1 - если у твитта положительная эмоциональная окраска и 0 - если отрицательная.

In [ ]:
df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x=='Positive' else 0)

In [ ]:
X = df.loc[:, df.columns != 'Sentiment']
y = df['Sentiment']

In [ ]:
X

,UserName,ScreenName,Location,TweetAt,OriginalTweet
0,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...
1,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...
2,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...
3,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV..."
4,3804,48756,"ÃÂT: 36.319708,-82.363649",16-03-2020,As news of the regionÃÂs first confirmed COV...
...,...,...,...,...,...
33439,44949,89901,OHIO,14-04-2020,I never that weÃÂd be in a situation &amp; w...
33440,44950,89902,NaN,14-04-2020,@MrSilverScott you are definitely my man. I fe...
33441,44952,89904,NaN,14-04-2020,Response to complaint not provided citing COVI...
33442,44953,89905,NaN,14-04-2020,You know itÃÂs getting tough when @KameronWi...


In [ ]:
y

0        1
1        1
2        1
3        0
4        1
        ..
33439    1
33440    0
33441    0
33442    1
33443    0
Name: Sentiment, Length: 33444, dtype: int64

In [ ]:
y.value_counts()

Sentiment
0    22022
1    11422
Name: count, dtype: int64

Сбалансированы ли классы?



---

**Ответ:** Как можно увидеть объектов класса = "0" практически в два раза больше, чем объектов класса = "1"

---



Выведете на экран информацию о пропусках в данных. Если пропуски присутствуют заполните их строкой 'Unknown'.

In [ ]:
np.sum(X.isna())

UserName            0
ScreenName          0
Location         7049
TweetAt             0
OriginalTweet       0
dtype: int64

In [ ]:
np.sum(y.isna())

0

In [ ]:
X['Location'].fillna("Unknown", inplace=True)
X

,UserName,ScreenName,Location,TweetAt,OriginalTweet
0,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...
1,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...
2,3802,48754,Unknown,16-03-2020,My food stock is not the only one which is emp...
3,3803,48755,Unknown,16-03-2020,"Me, ready to go at supermarket during the #COV..."
4,3804,48756,"ÃÂT: 36.319708,-82.363649",16-03-2020,As news of the regionÃÂs first confirmed COV...
...,...,...,...,...,...
33439,44949,89901,OHIO,14-04-2020,I never that weÃÂd be in a situation &amp; w...
33440,44950,89902,Unknown,14-04-2020,@MrSilverScott you are definitely my man. I fe...
33441,44952,89904,Unknown,14-04-2020,Response to complaint not provided citing COVI...
33442,44953,89905,Unknown,14-04-2020,You know itÃÂs getting tough when @KameronWi...


Разделите данные на обучающие и тестовые в соотношении 7 : 3 и `random_state=0`

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)

## Задание 2 Токенизация (3 балла)

Постройте словарь на основе обучающей выборки и посчитайте количество встреч каждого токена с использованием самой простой токенизации - деления текстов по пробельным символам и приведение токенов в нижний регистр

In [ ]:
import warnings

import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

warnings.filterwarnings("ignore")

In [ ]:
word_counts = {}
for tweet in X_train['OriginalTweet']:
    tokens = tweet.lower().split()
    for token in tokens:
        if token in word_counts:
            word_counts[token] += 1
        else:
            word_counts[token] = 1

print(list(word_counts.items())[:10])

[('why', 654), ('we', 3787), ('still', 763), ('want', 487), ('to', 23373), ('buy', 748), ('so', 1837), ('much', 482), ('stuff', 106), ('during', 1908)]


Какой размер словаря получился?

In [ ]:
len(word_counts), 1

(79755, 1)

Выведите 10 самых популярных токенов с количеством встреч каждого из них. Объясните, почему именно эти токены в топе

In [ ]:
best_10 = sorted(word_counts.items(), key=lambda item:item[1], reverse=True)


In [ ]:
best_10[:10]

[('.', 24118),
 (',', 17571),
 ('?', 9524),
 ('#coronavirus', 8808),
 ('â', 7415),
 ('\x82', 7311),
 ('19', 7167),
 ('-', 6643),
 ('covid', 6253),
 ('prices', 4601)]

**Ответ:** Получили топ, в котором самое большое количество упоминаний у знаков препинания, а после них сразу идет коронавирус, что логично поскольку весь датасет посвящен коронавирусу. Ковид и цены взаимосвязаны, так как остановилось практически все движения транспорта и люди практически не выходили из дома, что повлекло изменение цен как на продукты первой необходимости, так и на ценные активы и другие финансовые инструменты.



Удалите стоп-слова из словаря и выведите новый топ-10 токенов (и количество встреч) по популярности.  Что можно сказать  о нем?

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
nltk.download("stopwords", quiet=True)

True

In [ ]:
stop_words = set(stopwords.words('english'))

word_counts = {token: count for token, count in word_counts.items() if token not in stop_words}


In [ ]:
best_10 = sorted(word_counts.items(), key=lambda item:item[1], reverse=True)

In [ ]:
best_10[:10]

[('.', 24118),
 (',', 17571),
 ('?', 9524),
 ('#coronavirus', 8808),
 ('â', 7415),
 ('\x82', 7311),
 ('19', 7167),
 ('-', 6643),
 ('covid', 6253),
 ('prices', 4601)]

**Ответ:**  Теперь без стоп-слов топ рейтинга выглядит иначе. В нем находятся слова, которые связаны с темой коронавируса. Сам коронавирус встречается чаще всего, потому что такая тема, еда, цены, продовольствие и люди также находятся в топе, так как коронавирус на все это повлиял и люди это обсуждали в интернете.

Также выведите 20 самых непопулярных слов (если самых непопулярных слов больше выведите любые 20 из них) Почему эти токены непопулярны, требуется ли как-то дополнительно работать с ними?

In [ ]:
best_10[-40:]

[('legends!', 1),
 ("ciga's", 1),
 ('https://t.co/g5q23hv8qz', 1),
 ('film.', 1),
 ('amble', 1),
 ('telescope', 1),
 ('thc', 1),
 ('"definitely', 1),
 ('consequences,"', 1),
 ('@dralexamieses', 1),
 ('vapers,', 1),
 ('https://t.co/qdecroxk6p', 1),
 ('https://t.co/wojgzo5bsz', 1),
 ('https://t.co/2s1aljdwdd', 1),
 ('acquiring.', 1),
 ('https://t.co/pd6ual9yfi', 1),
 ('https://t.co/lw1r0rm7xs', 1),
 ('https://t.co/5cbliqzx7l', 1),
 ('now.when', 1),
 ('milion', 1),
 ('skellig', 1),
 ('coast!', 1),
 ('closer!)', 1),
 ('@skelligsix18', 1),
 ('#skelligcoast2kms', 1),
 ('#southkerry', 1),
 ('https://t.co/zjcl195vqs', 1),
 ('@srinivasiyc', 1),
 ('https://t.co/iaek4fwsgz', 1),
 ('premiership', 1),
 ('non-playing', 1),
 ('subsidise', 1),
 ('playersã\x82â\x92', 1),
 ('renewing', 1),
 ('wage!', 1),
 ('flew', 1),
 ('nothing...', 1),
 ('@torontopearson', 1),
 ('@680news', 1),
 ('https://t.co/7j2y3rsld9', 1)]

**Ответ:** Хорошие новости очевидно не могли быть в топе рейтинга по популярности, так как во время пандемии, особенно в начале не приходится надеятся на хорошее. Алкоголь, биткоин также ушли на второй план. Различные теги по-типу "!!" не являются сами по себе часто используемыми.


Поскольку деление происходила по пробелам, то мы получили токены не до конца разбитые, что несомненно говорит о том, что необходимо дополнительно их обработать. Поскольку они не до конца разбиты, то мы и получаем их редкое использование.






Теперь воспользуемся токенайзером получше - TweetTokenizer из библиотеки nltk. Примените его и посмотрите на топ-10 популярных слов. Чем он отличается от топа, который получался раньше? Почему?

In [ ]:
from nltk.tokenize import TweetTokenizer

In [ ]:
word_counts = {}
tw = TweetTokenizer()

In [ ]:
for tweet in X_train['OriginalTweet']:
    tokens = tw.tokenize(tweet.lower())
    for token in tokens:
        if token not in stop_words:
            if token in word_counts:
                word_counts[token] += 1
            else:
                word_counts[token] = 1

In [ ]:
best_10 = sorted(word_counts.items(), key=lambda item:item[1], reverse=True)

In [ ]:
best_10[:10]

[('.', 24118),
 (',', 17571),
 ('?', 9524),
 ('#coronavirus', 8808),
 ('â', 7415),
 ('\x82', 7311),
 ('19', 7167),
 ('-', 6643),
 ('covid', 6253),
 ('prices', 4601)]

**Ответ:** Топ TweetTokenizer ничем не отличается от топа обычного split()

Удалите из словаря стоп-слова и пунктуацию, посмотрите на новый топ-10 слов с количеством встреч, есть ли теперь в нем что-то не похожее на слова?

In [ ]:
from string import punctuation

noise = stopwords.words("russian") + list(punctuation)

In [ ]:
word_counts = {token: count for token, count in word_counts.items() if token not in noise}

In [ ]:
best_10 = sorted(word_counts.items(), key=lambda item:item[1], reverse=True)

In [ ]:
best_10[:10]

[('#coronavirus', 8808),
 ('â', 7415),
 ('\x82', 7311),
 ('19', 7167),
 ('covid', 6253),
 ('prices', 4601),
 ('\x92', 4372),
 ('food', 4367),
 ('store', 3877),
 ('supermarket', 3805)]

**Ответ:** Получлось так, что в топе присутствует токена типа "\x(номер)" и данный тип объектов не являются словами.

Удалите из словаря токены из одного символа, с позицией в таблице Unicode 128 и более (`ord(x) >= 128`)

Выведите топ-10 самых популярных и топ-20 непопулярных слов. Чем полученные топы отличаются от итоговых топов, полученных при использовании токенизации по пробелам? Что теперь лучше, а что хуже?

In [ ]:
token_counts = {token: count for token, count in token_counts.items() if len(token) > 1 or ord(token) < 128}

**Ответ:** # -- YOUR ANSWER HERE --

Выведите топ-10 популярных хештегов с количеством встреч. Что можно сказать о них?

In [ ]:
token_counts = {token: count for token, count in token_counts.items() if token[0] == '#'}

In [ ]:
summed_values = {key: sum(value) for key, value in token_counts.items()}

result = dict(summed_values)

In [ ]:
sorted(result.items(), key=lambda x:x[1], reverse=True)[:10]

**Ответ:** # -- YOUR ANSWER HERE --

То же самое проделайте для ссылок на сайт https://t.co Сравнима ли популярность ссылок с популярностью хештегов? Будет ли информация о ссылке на конкретную страницу полезна?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Используем опыт предыдущих экспериментов и напишем собственный токенайзер, улучшив TweetTokenizer. Функция tokenize должна:



*   Привести текст в нижний регистр
*   Применить TweetTokenizer для изначального выделения токенов
*   Удалить стоп-слова, пунктуацию, токены из одного символа, с позицией в таблице Unicode 128 и более и ссылки на t.co



In [ ]:
def custom_tokenizer(text):

  # -- YOUR CODE HERE --

  return tokens


In [ ]:
custom_tokenizer('This is sample text!!!! @Sample_text I, \x92\x92 https://t.co/sample  #sampletext')

## Задание 3 Векторизация текстов (2 балла)

Обучите CountVectorizer с использованием custom_tokenizer в качестве токенайзера. Как размер полученного словаря соотносится с размером изначального словаря из начала задания 2?

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer # -- YOUR CODE HERE --

print(len(cv.vocabulary_))

**Ответ:** # -- YOUR ANSWER HERE --

Посмотрим на какой-нибудь конкретный твитт:

In [ ]:
ind = 9023
train.iloc[ind]['OriginalTweet'], train.iloc[ind]['Sentiment']

Автор твитта не доволен ситуацией с едой во Франции и текст имеет резко негативную окраску.

Примените обученный CountVectorizer для векторизации данного текста, и попытайтесь определить самый важный токен и самый неважный токен (токен, компонента которого в векторе максимальна/минимальна, без учета 0). Хорошо ли они определились, почему?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Теперь примените TfidfVectorizer и  определите самый важный/неважный токены. Хорошо ли определились, почему?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Найдите какой-нибудь положительно окрашенный твитт, где TfidfVectorizer хорошо (полезно для определения окраски) выделяет важный токен, поясните пример.

*Подсказка:* явно положительные твитты можно искать при помощи положительных слов (good, great, amazing и т. д.)

In [ ]:
train[train['OriginalTweet'].apply(lambda x: 'your_good_word_here' in x) & (train['Sentiment'] == 1)]

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment


In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 4 Обучение первых моделей (1 балл)

Примените оба векторайзера для получения матриц с признаками текстов.  Выделите целевую переменную.

In [ ]:
# -- YOUR CODE HERE --

Обучите логистическую регрессию на векторах из обоих векторайзеров. Посчитайте долю правильных ответов на обучающих и тестовых данных. Какой векторайзер показал лучший результат? Что можно сказать о моделях?

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 5 Стемминг (0.5 балла)

Для уменьшения словаря можно использовать стемминг.

Модифицируйте написанный токенайзер, добавив в него стемминг с использованием SnowballStemmer. Обучите Count- и Tfidf- векторайзеры. Как изменился размер словаря?

In [ ]:
def custom_stem_tokenizer(text):
  # -- YOUR CODE HERE --
  return tokens

In [ ]:
custom_stem_tokenizer('This is sample text!!!! @Sample_text I, \x92\x92 https://t.co/sample  #sampletext adding more words to check stemming')

['sampl', 'text', '@sample_text', '#sampletext', 'ad', 'word', 'check', 'stem']

In [ ]:
cv = CountVectorizer # -- YOUR CODE HERE --

print(len(cv.vocabulary_))

36652


**Ответ** # -- YOUR ANSWER HERE --

Обучите логистическую регрессию с использованием обоих векторайзеров. Изменилось ли качество? Есть ли смысл применять стемминг?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание  6 Работа с частотами (1.5 балла)

Еще один способ уменьшить количество признаков - это использовать параметры min_df и max_df при построении векторайзера  эти параметры помогают ограничить требуемую частоту встречаемости токена в документах.

По умолчанию берутся все токены, которые встретились хотя бы один раз.



Подберите max_df такой, что размер словаря будет 36651 (на 1 меньше, чем было). Почему параметр получился такой большой/маленький?

In [ ]:
cv_df = CountVectorizer(tokenizer=custom_stem_tokenizer,
                        max_df=# -- YOUR CODE HERE --
                        ).fit(
                            # -- YOUR CODE HERE --
                            )
print(len(cv_df.vocabulary_))

36651


In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Подберите min_df (используйте дефолтное значение max_df) в CountVectorizer таким образом, чтобы размер словаря был 3700 токенов (при использовании токенайзера со стеммингом), а качество осталось таким же, как и было. Что можно сказать о результатах?

In [ ]:
cv_df = CountVectorizer(tokenizer=custom_stem_tokenizer,
                        min_df=# -- YOUR CODE HERE --
                        ).fit(
                            # -- YOUR CODE HERE --
                            )
print(len(cv_df.vocabulary_))

3700


In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

В предыдущих заданиях признаки не скалировались. Отскалируйте данные (при словаре размера 3.7 тысяч, векторизованные CountVectorizer), обучите логистическую регрессию, посмотрите качество и выведите `berplot` содержащий по 10 токенов, с наибольшим по модулю положительными/отрицательными весами. Что можно сказать об этих токенах?

In [ ]:
from sklearn.preprocessing import StandardScaler
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 7 Другие признаки (1.5 балла)

Мы были сконцентрированы на работе с текстами твиттов и не использовали другие признаки - имена пользователя, дату и местоположение

Изучите признаки UserName и ScreenName. полезны ли они? Если полезны, то закодируйте их, добавьте к матрице с отскалированными признаками, обучите логистическую регрессию, замерьте качество.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Изучите признак TweetAt в обучающей выборке: преобразуйте его к типу datetime и нарисуйте его гистограмму с разделением по цвету на оспнове целевой переменной. Полезен ли он? Если полезен, то закодируйте его, добавьте к матрице с отскалированными признаками, обучите логистическую регрессию, замерьте качество.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --



Поработайте с признаком Location в обучающей выборке. Сколько уникальных значений?

In [ ]:
# -- YOUR CODE HERE --

Постройте гистограмму топ-10 по популярности местоположений (исключая Unknown)

In [ ]:
# -- YOUR CODE HERE --

Видно, что многие местоположения включают в себя более точное название места, чем другие (Например, у некоторых стоит London, UK; а у некоторых просто UK или United Kingdom).

Создайте новый признак WiderLocation, который содержит самое широкое местоположение (например, из London, UK должно получиться UK). Сколько уникальных категорий теперь? Постройте аналогичную гистограмму.

In [ ]:
# -- YOUR CODE HERE --

Закодируйте признак WiderLocation с помощью OHE таким образом, чтобы создались только столбцы для местоположений, которые встречаются более одного раза. Сколько таких значений?


In [ ]:
# -- YOUR CODE HERE --

Добавьте этот признак к матрице отскалированных текстовых признаков, обучите логистическую регрессию, замерьте качество. Как оно изменилось? Оказался ли признак полезным?


*Подсказка:* используйте параметр `categories` в энкодере.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 8 Хорошее качество (Бонус 1 балл)

Добейтесь accuracy=0.9 на тестовой выборке (можно сменить токенайзер, векторайзер, модель и т.д.)

In [ ]:
# -- YOUR CODE HERE --